In [1]:
import os
import pandas as pd
import requests 
import csv
import json

In [2]:
# API endpoint
# NOTE: RxNorm has a "Prescribable" API endpoint for prescribable drugs only
# https://mor.nlm.nih.gov/RxNav/search?searchBy=RXCUI&searchTerm=258345
# https://lhncbc.nlm.nih.gov/RxNav/APIs/api-RxNorm.getAllConceptsByTTY.html
# https://lhncbc.nlm.nih.gov/RxNav/APIs/api-RxNorm.getTermTypes.html

################
# GET CONCEPTS TABLE
################
# Create URL for ingredients, main ingredients and brand names
URL = 'https://rxnav.nlm.nih.gov/REST/Prescribe/allconcepts.json?tty=IN+MIN+BN'

# send GET request and save the response as response object 
response = requests.get(url = URL) 

# extract data in json format 
data = response.json()

# Convert to df
df_concepts=pd.DataFrame(data['minConceptGroup']['minConcept'])

# Format data
df_concepts['name']=df_concepts['name'].str.lower()
df_concepts

,rxcui,name,tty
0,1117240,a+d diaper rash,BN
1,705892,a+d original,BN
2,203594,a-hydrocort,BN
3,203857,a-methapred,BN
4,605082,a.e.r. witch hazel,BN
...,...,...,...
10651,1670310,tipiracil / trifluridine,MIN
10652,214866,trandolapril / verapamil,MIN
10653,1009143,ubidecarenone / vitamin e,MIN
10654,1487518,umeclidinium / vilanterol,MIN


In [3]:
################
# GET RXCUIDS FOR NEXT STEP
################
prd_names=['adenocard', 'adenoscan', 'betmiga', 'myrbetriq', 'advagraf', 'astragraf', 'prograf']

rxcui_ids=df_concepts[df_concepts['name'].isin(prd_names)]['rxcui'].tolist()
rxcui_ids

['258345', '215025', '1300792', '196463']

In [4]:
################
# GET RXNORM RELATED INFO
################

def build_api_url(rxcui_id):
    return f"{BASE_URL}/REST/rxcui/{rxcui_id}/allrelated.json"

# Parameters
BASE_URL = "https://rxnav.nlm.nih.gov"

df_relconc = []
rxcui_id_arr = []

for rxcui_id in rxcui_ids:
    print(rxcui_id)
    
    # Build URL
    URL = build_api_url(rxcui_id)

    # send GET request and save the response as response object 
    response = requests.get(url = URL) 

    # extract data in json format 
    data = response.json()

    # Get related concepts
    data2=data['allRelatedGroup']['conceptGroup']

    # Appending rows
    for i in range(1,len(data2)):
        tty=data2[i]['tty']
        try:
            q=pd.DataFrame(data2[i]['conceptProperties'])
        except:
            q=None

    
        if (q is not None):
            q['rxcui_id']=rxcui_id
            df_relconc = pd.DataFrame(df_relconc).append(q, ignore_index=True)
        
df_relconc

258345
215025
1300792
196463


,rxcui,name,synonym,tty,language,suppress,umlscui,rxcui_id
0,721656,Prefilled Syringe,,DF,ENG,N,,258345
1,1151126,Injectable Product,,DFG,ENG,N,,258345
2,296,adenosine,,IN,ENG,N,,258345
3,727357,4 ML adenosine 3 MG/ML Prefilled Syringe [Aden...,Adenocard 12 MG per 4 ML Prefilled Syringe,SBD,ENG,N,,258345
4,727361,2 ML adenosine 3 MG/ML Prefilled Syringe [Aden...,Adenocard 6 MG per 2 ML Prefilled Syringe,SBD,ENG,N,,258345
...,...,...,...,...,...,...,...,...
93,374015,tacrolimus Oral Capsule,,SCDF,ENG,N,,196463
94,1163655,tacrolimus Injectable Product,,SCDG,ENG,N,,196463
95,1164202,tacrolimus Oral Product,,SCDG,ENG,N,,196463
96,1164203,tacrolimus Pill,,SCDG,ENG,N,,196463


In [5]:
df_relconc[df_relconc['tty']=='IN']

,rxcui,name,synonym,tty,language,suppress,umlscui,rxcui_id
2,296,adenosine,,IN,ENG,N,,258345
15,296,adenosine,,IN,ENG,N,,215025
31,1300786,mirabegron,,IN,ENG,N,,1300792
61,42316,tacrolimus,,IN,ENG,N,,196463
